# Optimisation - speeding up your code
## Martin Robinson
## Nov 2019

# Optimisation

- The Zen of Python (PEP 20):
  - Beautiful is better than ugly.
  - Explicit is better than implicit.
  - Simple is better than complex.
  - Complex is better than complicated.

- Python's design philisophy emphasises code readability and simplicity over performance

- Often in scientific computing we wish to improve the performance of our code

# What to optimise - profiling


    "We should forget about small efficiencies, say about 97% of the time: premature 
    optimization is the root of all evil. Yet we should not pass up our opportunities in 
    that critical 3%" - Donald Knuth (1974)

- Design and implement your code first! Then you can measure where the program spends 
  most of its time (i.e. "that critical 3%") and optimise them accordingly.

# The cProfile module

- part of the standard library 
  - <https://docs.python.org/3.2/library/profile.html>
- Implements *Deterministic Profiling*. Monitors the entry and exit from each function. 
  Gathers statistics on, e.g., number of times called and time spent in every function 
  called

In [ ]:
import time
import cProfile

def slow_function():
  time.sleep(1)

def fast_function():
  time.sleep(0.1)

def profiled_program():
  slow_function()
  fast_function()

cProfile.run('profiled_program()', sort='cumulative')

In [ ]:
%%prun -s cumulative
profiled_program()

# The timeit module

- Deterministic profiling adds overhead to the code being profiled, not useful for 
  benchmarking
- Use a timer instead. Python has the `timeit` module
  - <https://docs.python.org/3.6/library/timeit.html>
- `timeit` uses the `time.perf_counter` timer internally
  - <https://docs.python.org/3.6/library/time.html#time.perf_counter>

In [ ]:
import timeit

def slow_function():
  return sum([i for i in range(1000)])

def fast_function():
  return sum([i for i in range(100)])


In [ ]:
timeit.Timer('fast_function()', setup="from __main__ import fast_function").repeat(repeat=3, number=10)

In [ ]:
timeit.Timer('slow_function()', setup="from __main__ import slow_function").repeat(repeat=3, number=10)

In [ ]:
import time
start_time = time.perf_counter()
fast_function()
end_time = time.perf_counter()
print('fast function took {} seconds'.format(end_time-start_time))

In [ ]:
%%writefile tmp.py
def fast_function():
  return sum([i for i in range(100)])
fast_function()

In [ ]:
%%bash
python -m timeit 'from tmp import fast_function; fast_function()'

In [ ]:
%%timeit
fast_function()

# Methods for optimising Python code 

1. Rewrite using data structures / algorithms
  - most of the (time-consuming) code you write has already been implemented as commonly 
    used data structures and algorithms, take the time to know these and how to use 
    them.
1. Use a high-performance library
  - Many (most?) python libraries are written in a faster compiled language (e.g. 
    C/C++/fortran), offering significant speed-up
  - For scientific programming, [`numpy`](https://numpy.org/) and 
    [`scipy`](https://www.scipy.org/) offer many possibilities
    

3. "Compile" Python code to a low-level language (usually C/C++).
  - [`cython`](https://cython.org/): add static type declarations to your code and 
    compile to C or C++
  - [`numba`](https://numba.pydata.org/): automatically compiles python code directly to 
    machine code.
  - [`theano`](http://deeplearning.net/software/theano/): compiles mathematical 
    expressions involving numpy-like arrays to C. Largely replaced now by 
    [`tensorflow`](https://www.tensorflow.org/) and [`pytorch`](https://pytorch.org/).
1. Write your code in a low-level language and call it from Python
  - Many "wrapper" libraries exist, such as [`boost 
    python`](https://www.boost.org/doc/libs/1_70_0/libs/python/doc/html/index.html) or 
    [`pybind11`](https://github.com/pybind/pybind11) for C++, or 
    [`swig`](http://www.swig.org) for C/C++, and 
    [`f2py`](https://docs.scipy.org/doc/numpy/f2py/) for fortran

5. Parallise your Python code
  - Python is difficult to run in parallel due to the Global Interpreter Lock (GIL). 
    Within a single process different python threads are restricted to run in serial.
  - However, it is possible to run many communicating python processes using the 
    `multiprocessing` module

# Numpy vectorisation

- each numpy operation is written in optimised C, and compiled to machine code. 
- speed up your python loops by converting them to numpy operations where possible
- this process is known as *vectorisation*, also a big issue in Matlab

In [ ]:
import numpy as np

def python_sum(array):
  s = 0
  for i in range(len(array)):
    s += array[i]
  return s

def numpy_sum(array):
  return np.sum(array)

In [ ]:
n = 1000
%timeit python_sum(np.ones(n))

In [ ]:
%timeit numpy_sum(np.ones(n))

# examples - array creation

- numpy has many functions for generating arrays
    - `np.empty`, `np.full`, `np.zeros`, `np.ones`, `np.eye`, `np.linspace`, `np.random.random`

In [ ]:
def python_create():
  return [i**2 for i in range(n)]

def numpy_create():
  return np.arange(0,n,1)**2

In [ ]:
%timeit python_create()

In [ ]:
%timeit numpy_create()

# examples - single loops

- replace single loops over arrays into the equivilent number operations
- example: $v_i + a \exp(-v_i)$

In [ ]:
import math
a = 1.5
def python_expr(array):
  for i in range(len(array)):
    array[i] += a * math.exp(-array[i])

def numpy_expr(array):
  print('what goes here?')

In [ ]:
def numpy_expr(array):
  array + a * np.exp(-array)

In [ ]:
data = 1e-4*np.ones(1000)
%timeit python_expr(data)

In [ ]:
data = 1e-4*np.ones(1000)
%timeit numpy_expr(data)

# examples - broadcasting

- Using the numpy broadcasting rules can help when vectorising

In [ ]:
a = [1.5, 2.5]

def python_2d_expr(array):
  for i in range(array.shape[0]):
    for j in range(array.shape[1]):
      array[i, j] += a[j] * math.exp(-array[i, j])

np_a = np.array(a)

def numpy_2d_expr(array):
  print('what goes here?')

In [ ]:
def numpy_2d_expr(array):
  array + np_a * np.exp(-array)

In [ ]:
data = 1e-4*np.ones((1000,2))
%timeit python_2d_expr(data)

In [ ]:
data = 1e-4*np.ones((1000,2))
%timeit numpy_2d_expr(data)

# examples - more broadcasting - double loops

- Even double loops can be vectorised with appropriate broadcasting

In [ ]:
n = 100
a = np.abs(np.random.randn(n))
b = np.abs(np.random.randn(n))

def python_dbl_loop(array):
  for i in range(array.shape[0]):
    for j in range(array.shape[1]):
      array[i, j] += math.exp(- (a[i] + b[j])) 
      
np_a = np.array(a)
np_b = np.array(b)

def numpy_dbl_loop(array):
  print('what goes here?')

In [ ]:
def numpy_dbl_loop(array):
  array + np.exp(a + np.transpose(b))

In [ ]:
data = 1e-4*np.ones((n,n))
%timeit python_dbl_loop(data)

In [ ]:
data = 1e-4*np.ones((n,n))
%timeit numpy_dbl_loop(data)

# The practical - speeding up a cell model

The practical exercises over the next 2 days revolve around speeding up a Python 
implementation of a individual-based model of diffusing and interacting cells.

The model consists of a set of $N$ cells in a periodic unit square domain. Let
$\mathbf{X}_i(t)$ denote the position of the $i$th particle in $\Omega \subset \mathbb R^2$.
For each particle $i$, the motion through space is described by a stochastic
differential equation (SDE).


$$
\mathrm{d}\mathbf{X}_i(t) = \sqrt{2 D_\alpha} \mathrm{d}\mathbf{W}_i(t) - \sum_{j\ne i} \nabla_i u(\| \mathbf{X}_i(t) - \mathbf{X}_j(t) \|) \mathrm{d}t,
$$

where $D=1$ is the diffusion coefficient of the cells, and $\nabla_i$ denotes the
gradient with respect to $\mathbf{X}_i$. The interaction potential $u$ may be a soft potential
incorporating effects such as size exclusion by cells and cell-cell adhesion. In this
case we use the soft exponential potential.

The simple method to numerically integrate the SDE is to use a fixed time-step $\Delta 
t$ and a Euler--Maruyama discretisation, resulting in the time-stepping scheme:

$$ 
\mathbf{X}_i(t+ \Delta t) = \mathbf{X}_i(t) + \sqrt{2D_\alpha \Delta t} \xi_i - \sum_{j\ne i} \nabla_i u(\| \mathbf{X}_i(t) - \mathbf{X}_j(t) \|) \Delta t, 
$$

where $\xi_i$ is a two-dimensional normally distributed random variable with zero mean
and unit variance.

# Implementation

[walk through code here]